> # Superstore Sales forecasting

# Problem Statement:
* To increase the Superstore sales, superstore gave us a task to understand what segments, regions, cities or customers they should be focusing on. The goal is to give practical insights to improve profit based on sales.
* In the dynamic world of retail, forecasting sales accurately is a critical aspect of optimizing operations, managing inventory, and ensuring profitability. This project revolves around a retail dataset spanning four years from a global superstore. This will done using EDA and Regression Technique


# Objective
* To Perform EDA and Predict the sales by using Regression ML algorithms!


# Solution:
* Perform EDA (Data Analysis,Data Visualization, Data Cleaning) to understand the data and to clean the dataset for further process
* Perform data preprocessing
* Build pipeline to train  and test the ML Regression model
* Model Evaluation
* Conclusion

**Dataset Columns:**
* Row ID : 'Unique Order ID for each Customer'
* Order ID: 'Unique ID for each row',
* Order Date: 'Order Date of the product',
* Ship Date: 'Shipping Date of the Product',
* Ship Mode: 'Shipping Mode specified by the Customer',
* Customer ID: 'Unique ID to identify each Customer',
* Customer Name: 'Name of the Customer',
* Segment: 'The segment where the Customer belongs',
* Country: 'Country of residence of the Customer',
* City: 'City of residence of of the Customer',
* State: 'State of residence of the Customer',
* Postal Code: 'Postal Code of every Customer',
* Region: 'Region where the Customer belong',
* Product ID: 'Unique ID of the Product',
* Category: 'Category of the product ordered',
* Sub-Category: 'Sub-Category of the product ordered',
* Product Name: 'Name of the Product',
* Sales: 'Sales of the Product'

# Packages

In [ ]:
# !pip install prophet

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import time


import warnings
warnings.filterwarnings("ignore")

# Data

In [ ]:
df = pd.read_csv('/kaggle/input/sales-forecasting/train.csv')
df.head()

# Exploratory Data Analysis

In [ ]:
df.info()

## Handle missing values

* As here you can see Postal Code column having 11 missing values
* 1st we can check relation between Country,City,State with Postal Code so, if possible then to treat the missing value we can replace it appropiate/ respective values

In [ ]:
df.loc[df["Postal Code"].isna(),['Country','City','State','Postal Code']]

In [ ]:
df.loc[(df['City']=='Burlington') & (df['State']=='Vermont'),['Country','Postal Code']]

**Observation**
* Here you can see where State is Vermount and City is Burlington we don't have any Postal Code in this dataset
* So we have two option to deal with this
    1. Remove all missing data
    2. Check Postal Code for this State and City online and place that value here
* As we know how much data are important so 1st we'll try to search Postal Code online and try to use that

**Search Result**
* You can check for postal code of burlington city in vermont for shipping & you'll get the search result as :
            **ZIP Code 05401**
* So here to deal with missing value we are going to use 05401
* Before performing any operation on the data set first you need to create a copy and then do all the operation.

In [ ]:
train = df.copy()
train['Postal Code'] = train['Postal Code'].fillna(5401) # leading zeros in decimal integer literals are not permitted so we use 5401

In [ ]:
train.isna().sum().sum()

## Drop Unwanted Features

* As you can see there is no use of Row ID, Order ID, Customer ID and Customer Name to train our model so just remove these.

In [ ]:
train.drop(['Row ID','Customer Name','Order ID','Product ID'],axis=1,inplace=True)

## Check and drop Duplicate data

In [ ]:
train[train.duplicated()]

In [ ]:
train.drop_duplicates(inplace=True)
train.duplicated().sum()

## Convert Order date and ship date dtype object to datetime and Postal Code float to int

* To play with dates first need to convert these columns with datetime data type

In [ ]:
train['Order Date'] = pd.to_datetime(train['Order Date'], format='%d/%m/%Y')
train['Ship Date'] = pd.to_datetime(train['Ship Date'], format='%d/%m/%Y')
train['Postal Code'] = train['Postal Code'].astype(int)

* Here we seperate out the month, year,dates for further analysis

In [ ]:
train.insert(loc=4,  column='order_month_year',value=train['Order Date'].dt.to_period('M'))
train.insert(loc=5, column='ship_month_year', value=train['Ship Date'].dt.to_period('M'))

train.insert(loc=6, column='order_day', value=train['Order Date'].dt.day)
train.insert(loc=7, column='order_month', value=train['Order Date'].dt.month)
train.insert(loc=8, column='order_year', value=train['Order Date'].dt.year)

train.insert(loc=9, column='ship_day', value=train['Ship Date'].dt.day)
train.insert(loc=10, column='ship_month', value=train['Ship Date'].dt.month)
train.insert(loc=11, column='ship_year', value=train['Ship Date'].dt.year)

In [ ]:
train.info()

In [ ]:
pd.set_option('display.max_columns', None)
train.head(3)

## Univariate

## Target variable

In [ ]:
train.Sales.describe()

In [ ]:
sns.displot(train['Sales'],kde=True,aspect=2,color='slategray',bins=100);

**Observation**
* Approx Product Sales are 12K to 26K

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(train['Sales'])

## Categorical data

**Ordinal Data**

In [ ]:
color_list = ['slategray','darkslategray','steelblue','teal','cadetblue','powderblue']

train['Ship Mode'].value_counts().plot.bar(color = color_list)
plt.xticks(rotation=360);

**Observation**
* 5859 orders were shipped by Standrad Class ship mode

## Categorical data

**Nominal Data**

In [ ]:
train['Segment'].value_counts().plot.bar(color = color_list)
plt.xticks(rotation=360);

**Observation**
* Approx 52% of segments for product sales was Consumers

In [ ]:
plt.figure(figsize=(25,4))
train['State'].value_counts().plot.bar(color = color_list)
plt.xticks(rotation=60);

**Observation**
* Approx 20% order was placed by State California             

In [ ]:
plt.figure(figsize=(25,4))
train['City'].value_counts().head(25).plot.bar(color = color_list)
plt.xticks(rotation=40);

**Observation**
* New York City citizen were contributing approx 9% in overall product sales

In [ ]:
train['Region'].value_counts().plot.pie(subplots=True,autopct='%1.0f%%',colors=color_list);

**Observation**
* 32% customers are Western Region of United State where as 16% are from Southern Region  

In [ ]:
train['Category'].value_counts().plot.pie(subplots=True,autopct='%1.0f%%',colors=color_list);

**Observation**
* 60% of product was sale are belongs to Office Supplies Category

In [ ]:
plt.figure(figsize=(25,7))
train['Sub-Category'].value_counts().plot.bar(color = color_list)
plt.xticks(rotation=360);

**Observation**
* In overall sub-category approx 15% are Binders

In [ ]:
plt.figure(figsize=(12,6))
train['Product Name'].value_counts().head(25).sort_values().plot.barh(color = 'Slategrey')
plt.xlabel('Count');

In [ ]:
plt.figure(figsize=(12,7))
train['Product Name'].value_counts().sort_values(ascending=False).tail(25).plot.barh(color = 'Slategrey');

**Observation**
* Staple envelope, Staples, Easy-staple paper are most ordered product                                                   

* Xiaomi Mi3, Universal Ultra Bright White Copier/Laser Paper, 8 1/2" x 11", Ream, Socket Bluetooth Cordless Hand Scanner (CHS), Logitech Illuminated Ultrathin Keyboard with Backlighting, LG G2 are less orderd product


In [ ]:
train['Shipping_days'] = train['Ship Date'] - train['Order Date']
time_taken = train['Shipping_days'].value_counts().sort_values().plot.barh(color=color_list)
plt.xlabel('count')


**Observation**
* The Maximum days taken for the shipping after order was placed is for 4 days

# Bivariate or Multivariate

Features v/s Target variable

In [ ]:
top_customer = train.groupby("Customer ID").agg({'Sales':sum}).sort_values(by= "Sales",ascending=False).head(10).plot.barh(color= color_list)
plt.gca().invert_yaxis()
plt.xlabel('Sales');

**Observation:**
* Here the top 10 Customer who bought the product maybe at high amount or product which have high cost here Customer ID SM-20320 is on top

In [ ]:
top_States= train.groupby("State").agg({'Sales':sum}).sort_values(by= "Sales",ascending=False).round(2).head(15).plot.barh(color= color_list)
plt.gca().invert_yaxis()
plt.xlabel('Sales');

**Observation**
* Above chart shows top 15 States from which California is the on the top in terms of the Sales

In [ ]:
top_cities= train.groupby("City").agg({'Sales':sum}).sort_values(by= "Sales",ascending=False).round(2).head(15).plot.barh(color='slategray')
plt.gca().invert_yaxis()
plt.xlabel('Sales');

**Observation**
* The above bar chart shows top 15 cities out of 529 cities based on their Sales
* Most of the sales are done by New York City

In [ ]:
train.loc[train['State']=='California',['City','Sales']].groupby('City')['Sales'].sum().sort_values(ascending= False).head(15).plot.barh(color='Slategray');
plt.gca().invert_yaxis()
plt.xlabel('Sales');

**Observation**
* As we saw in above bar chart California State is having most Sales so lets have look which cities are contributed to it.
* Out of 80 cities in California here are top 15 cites from which Los Angeles have most sales.

In [ ]:
train.groupby("Ship Mode").agg({'Sales':sum}).plot.pie(subplots=True, explode = [0,0,0,0.2],autopct='%1.0f%%',colors=['slategray','steelblue','teal','darkslategray'])

**Observation**
* 59% Standard Class ship mode are used, or we can say Standard Class ship mode is on high demand

In [ ]:
train.groupby('Segment').agg({'Sales':sum}).plot.pie(subplots=True, explode = [0.1,0,0],autopct='%1.0f%%',colors=['slategray','steelblue','darkslategray']);

**Observation**
* 51% of the sales was from Consumer

In [ ]:
train.groupby('Region').agg({'Sales':sum}).plot.pie(subplots=True, explode = [0,0,0,0.2],autopct='%1.0f%%',colors=['slategray','steelblue','teal','darkslategray']);

**Observation**
* West region gave 31% of product order

In [ ]:
train.groupby('Category').agg({'Sales':sum}).plot.pie(subplots=True, explode = [0,0,0.1],autopct='%1.0f%%',colors=['slategray','steelblue','darkslategray']);

**Observation**
* 37% of the product sales price was from Technological product

In [ ]:
train.groupby('Sub-Category').agg({'Sales':sum}).sort_values(by= "Sales",ascending=False).round(2).plot.barh(color='slategray');
plt.gca().invert_yaxis()
plt.xlabel('Sales');

**Observation**
* Phones are the top demanded product in sub-category

In [ ]:
train.loc[train['Sub-Category']=='Phones',['Product Name','Sales']].groupby('Product Name')['Sales'].sum().sort_values(ascending= False).head(15).plot.barh(color='Slategray');
plt.gca().invert_yaxis()
plt.xlabel('Sales')

**Observation**
* Here we can say Samsung Galaxy Mega 6.3 was not most ordered product but still it contributed more in selling price of the product

### Top product names from all sub-category

In [ ]:
train['rank'] =train.groupby(['Sub-Category'])['Sales'].rank('dense',ascending=False)

In [ ]:
pd.DataFrame(train.loc[train['rank']==1,['Sub-Category', 'Product Name']].values,columns=['Sub-Category','Top Product name from each Sub-Category'])

In [ ]:
color_month = ['lightsteelblue','powderblue','slategray','cadetblue', 'teal', 'lightseagreen',
             'mediumseagreen','mediumaquamarine','cadetblue','deepskyblue', 'steelblue','darkslategray']

monthly_sales = pd.DataFrame(train.groupby('order_month_year')['Sales'].sum()).reset_index()
plt.figure(figsize=(20, 5))
sns.barplot(x = 'order_month_year', y = 'Sales',data = monthly_sales,palette=color_month)
plt.xticks(rotation=90)
plt.show()

**Observation:**
* From the above plot we can say that in every year September, November and December were the months having most sales
* Let's check the same with the product category


In [ ]:
sale = train.groupby(['order_year','order_month','Category']).agg({'Sales':sum}).reset_index()
sns.catplot(data = sale, kind="bar",x="Category", y="Sales",col='order_year',hue='order_month',height=5, aspect=1, palette=color_month);

**Observation:**
* As you observed in every year most profit related to Furniture happened in the month of 9,11,12 i.e., Sept,Nov,Dec resp.
* Whereas in each year in the month of 9 and 11 most of the sales profit done from the Office Supplies category products but only in year 2017 Dec also included in this profitable month
* But regarting to Technological products its varies in every year but overall we can say that in the 1st quarter of the year month 3 and in 3rd quarter month 9 and whole 4th quarter of the year are more favourable of the sales

In [ ]:
sale = train.groupby(['Region','Category','Segment']).agg({'Sales':sum}).reset_index()
sns.catplot(data = sale, kind="bar",x="Category", y="Sales", col="Region",hue='Segment',height=4, aspect=1, palette=color_list[0:3]);

**Observation**
* As you can see the Consumer were responsible for most of the sales of each product category in every region.
* The sales of technological products is highest in the western and eastern regions, followed by furniture then office Supplies.
* Whereas in Central and Southern region sales of Furniture and office Supplies are more tha the Technological products.

In [ ]:
sns.heatmap(train.corr(numeric_only=True),annot=True);

**Observation:**
* The highest correlation is between the order year and shipping year followed by the order month and shipping month which may the signified that the delay rate of shipping is much less

**Conclusion from the Data Visualization:**
* We can say that more profitable region is West and East whereas New york and California having highest profitable states.
* and in terms of the Product Category Technology is highest but Furniture and Office Supplier are also good there are so many demand in all these product category
* And in terms of the month the 9,11,12 are most preferable month maybe due to festive season or holidays

## Time Series Analysis

In [ ]:
regions = train['Region'].unique()
shipModes = train['Ship Mode'].unique()
categories = train['Category'].unique()

In [ ]:
plt.figure(5, figsize=(20, 6))
plt.subplots_adjust(wspace=0.1);
#graphing 'Sales' vs. 'Category'
dfbyCatOrderDate = train.groupby(['Category','Order Date'])
plt.subplot(131).title.set_text('Cumulative Number of Sales per Category')
plt.xticks(rotation=90)
plt.legend()
for cat in categories:
    data = dfbyCatOrderDate['Sales'].count()[cat]
    data = data.cumsum()
    sns.lineplot(data=data, ax=plt.subplot(131))

#graphing 'Sales' vs. 'Ship Mode'
dfbyShipOrderDate = train.groupby(['Ship Mode','Order Date'])
plt.subplot(132).title.set_text('Cumulative Number of Sales per Ship Mode')
plt.xticks(rotation=90)
plt.legend()
for shipMode in shipModes:
    data = dfbyShipOrderDate['Sales'].count()[shipMode]
    data = data.cumsum()
    sns.lineplot(data=data, ax=plt.subplot(132))

#graphing 'Sales' vs. 'Region'
dfbyRegionOrderDate = train.groupby(['Region','Order Date'])
plt.subplot(133).title.set_text('Cumulative Number of Sales per Region')
plt.xticks(rotation=90)
plt.legend()
for region in regions:
    data = dfbyRegionOrderDate['Sales'].count()[region]
    data = data.cumsum()
    sns.lineplot(data=data, ax=plt.subplot(133))

**Observation**
* As year passes sales are increasing

## Drop Unwanted Features

* As you can see there is Customer ID to train our model so just remove these.
* As we have Country, City, State and, Postal Code so insteading of keeping all of these we just keep Postal Code.
* Shipping_days and rank columns were just created for the analysis purpose so here I'm dropping these columns.

In [ ]:
train.drop(['Customer ID','Country','City','State','Shipping_days','rank'],axis=1,inplace=True)

In [ ]:
pd.set_option('display.max_columns',None)
train.sample()

In [ ]:
train.shape

## Handling Outliers:
* As we already see in the Data Visualization part, the Sales column having some outliers so it is important to handle this  

In [ ]:
train['Sales_log'] = np.log(train['Sales'])

In [ ]:
fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(121)
stats.probplot(train['Sales'], dist="norm", plot=ax1)
ax1.set_title('Outliers before Log transformation')
ax2 = fig.add_subplot(122)
stats.probplot(train['Sales_log'],dist="norm", plot=ax2)
ax2.set_title('Outliers after Log transformation')
plt.show()

In [ ]:
pd.set_option('display.max_columns',None)
train.sample(2)

## Data Encoding

* Before going to train the model we need to convert object data into appropiated numeric values by using some of the Data encoding techniques

### Feature mapping

In [ ]:
train['Ship Mode_encoded'] = train['Ship Mode'].replace(['Standard Class', 'First Class', 'Second Class', 'Same Day'],[1,2,3,4])
train.sample()

### Label Encoding

In [ ]:
X = train.drop(['Ship Mode','Order Date', 'order_month_year', 'Ship Date', 'ship_month_year', 'Sales', 'Sales_log'], axis=1)
for col in X.columns:
    if X[col].dtype == 'object':
        label = LabelEncoder()
        X[col] = label.fit_transform(X[col].values)

y = train['Sales_log']

In [ ]:
X.sample()

# Data Spliting

* Split the data into two parts for training and testing purpose

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

X_train.shape,y_train.shape,X_test.shape,y_test.shape

# Modeling

In [ ]:
class Regression_Models:
    def __init__(self,model_list,X_train,y_train,X_test,y_test):
        self.model_list = model_list
        self.X_train= X_train
        self.y_train= y_train
        self.X_test = X_test
        self.y_test = y_test

    def pipelining(self,model):
        p1=Pipeline([('sc',StandardScaler()),('model',model)])
        p1.fit(X_train,y_train)
        pred = p1.predict(X_test)
        decor = '_'*30
        print(f'{decor}{type(model).__name__}{decor}\n\nTrain: {p1.score(X_train,y_train)},\
        \nTest: mse:{mean_squared_error(y_test,pred)}, r2:{r2_score(y_test, pred)}\n\n')

    def models(self):
        for model in self.model_list:
            self.pipelining(model)

In [ ]:
model_list = [RandomForestRegressor(),LinearRegression(), Ridge(), KNeighborsRegressor(),
             Lasso(),SVR(),DecisionTreeRegressor(),XGBRegressor()]

default_parameter = Regression_Models(model_list,X_train,y_train,X_test,y_test)
default_parameter.models()

## Feature importance and selection

In [ ]:
rfr = RandomForestRegressor(100)
rfr.fit(X_train,y_train)

In [ ]:
feature_importance = pd.DataFrame({'importance': rfr.feature_importances_},
                                  index= X.columns).sort_values('importance')
feature_importance.plot.barh(color=color_list);

In [ ]:
X_imp =X[feature_importance[feature_importance['importance'] > 0.05].index]
X_imp

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_imp,y,test_size=0.3,random_state=0)

In [ ]:
feature_select_ = Regression_Models(model_list,X_train,y_train,X_test,y_test)
default_parameter.models()

## Hyperparameter tunning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
hyperparameter_model_list = [KNeighborsRegressor(),DecisionTreeRegressor(),
                             Lasso(),Ridge(),RandomForestRegressor()]


hyperparameters = [{'metric' : ['euclidean','minkowski','manhattan'],
                      'n_neighbors' :range(3,11)},
                 {"max_depth": range(3,6),
                     "max_features": range(1,11),
                     "min_samples_split": range(2,11),
                     "criterion": ["squared_error", "friedman_mse", "absolute_error"]},
                 {"alpha":[0.001,0.01,0.1,1,10,1000]},
                 {'alpha':[0.001,0.01,0.1,1,10,1000]},
                 {"n_estimators":[50,100,200],
                               "max_depth": range(3,6),
                               "max_features": range(1,11),
                               "min_samples_split": range(2,11)}]


In [ ]:
def randomized(hyperparameter_model_list,hyperparameters,cv=5):

    for model,hyperparameter in zip(hyperparameter_model_list,hyperparameters):

        random_search = RandomizedSearchCV(model,param_distributions=hyperparameter,cv=cv,n_iter=10)
        random_search.fit(X_train,y_train)

        pred = random_search.predict(X_test)

        print(f'{type(model).__name__}\n\ntrain score : {random_search.score(X_train,y_train)} ,\
        \ntest score : mse:{mean_squared_error(y_test,pred)}, r2:{r2_score(y_test, pred)}\n')

        print(random_search.best_params_,'\n\n')

In [ ]:
randomized(hyperparameter_model_list,hyperparameters)

**Conclusion**
* After applying hyperparameter tunning the model's performance not improved
* But feature selection with default parameters works good
* Overall We can say that RandomForestRegressor gives good performance specially with default parameters on Selected features.